In [70]:
import numpy as np
import pandas as pd
import h5py
import scipy.sparse as sps
from datetime import timedelta
import datetime

# pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.display.max_columns = None
pd.options.display.max_rows = 100
pd.options.display.min_rows = 50
pd.options.display.precision = 2
pd.options.display.expand_frame_repr = True


In [71]:
filename = '../data/all_hourly_data.h5'

In [72]:
with h5py.File(filename, "r") as f:
    print("Keys: %s" % f.keys())


Keys: <KeysViewHDF5 ['codes', 'interventions', 'patients', 'vitals_labs', 'vitals_labs_mean']>


In [73]:
patients = pd.read_hdf(filename, key="patients")

one_hot = pd.get_dummies(patients[["gender", "admission_type", "first_careunit"]])
patients = patients.join(one_hot)
#display(patients.columns)

# include only patients with 6h < los < 600h or same for los_icu, but los_icu is in days
LOS_LOWER_THRESH = 6 #exclude if LOS is shorter than this many hours
LOS_UPPER_THRESH = 600 #exclude if LOS is longer than this many hours
patients["los"] = (patients["dischtime"] - patients["admittime"])
patients = patients[((patients["los"] > timedelta(hours=LOS_LOWER_THRESH))  & (patients["los"] < timedelta(hours=LOS_UPPER_THRESH))) | 
                    ((patients["los_icu"] * 24 > LOS_LOWER_THRESH)            & (patients["los_icu"] * 24 < LOS_UPPER_THRESH))]

MIN_AGE_THRESH = 18
patients = patients[patients["age"] > MIN_AGE_THRESH]

#fix ages set to 300+ for anonymity purposes
patients.loc[patients['age'] > 90, 'age'] = 90

# ignore following dummies, common practice to exclude 1 dummy per category
# gender_M
# admission_type_ELECTIVE
# first_careunit_CCU

patients = patients[["age", "gender_F", "admission_type_EMERGENCY", "admission_type_URGENT", "first_careunit_CSRU", "first_careunit_MICU", "first_careunit_SICU", "first_careunit_TSICU"]]
patients.columns = ['age', 'is_F', 'is_emergency', 'is_urgent', 'is_csru', 'is_micu', 'is_sicu', 'is_tsicu']

# normalize
patients = ( patients - patients.mean() ) / patients.std()

print(patients.shape)
display(patients.head())


(37543, 8)


,,,age,is_F,is_emergency,is_urgent,is_csru,is_micu,is_sicu,is_tsicu
subject_id,hadm_id,icustay_id,,,,,,,,
3,145834,211552,0.72,-0.88,0.48,-0.17,-0.5,1.35,-0.44,-0.39
4,185777,294638,-0.92,1.14,0.48,-0.17,-0.5,1.35,-0.44,-0.39
6,107064,228232,0.12,1.14,-2.09,-0.17,-0.5,-0.74,2.26,-0.39
9,150750,220597,-1.26,-0.88,0.48,-0.17,-0.5,1.35,-0.44,-0.39
11,194540,229441,-0.79,1.14,0.48,-0.17,-0.5,-0.74,2.26,-0.39


In [74]:
interventions = pd.read_hdf(filename, key="interventions")
interventions["vasopressor"] = interventions[['vaso', 'vasopressin']].any(axis='columns').astype(int)
interventions = interventions[["vasopressor"]]
print(interventions.shape)
display(interventions.head())

(3183638, 1)


vasopressor
subject_id hadm_id icustay_id hours_in             
3          145834  211552     0                   0
                              1                   1
                              2                   1
                              3                   1
                              4                   1

In [75]:
# exclude patients that get vasopressor in the first 6h e.g. 0-5
excluded_subject_ids = interventions[interventions.index.get_level_values("hours_in").isin(range(6)) & interventions["vasopressor"] == 1]
excluded_subject_ids = excluded_subject_ids.index.get_level_values("subject_id").unique()
excluded_subject_ids.shape

(8406,)

In [76]:
outcome = pd.DataFrame(interventions.groupby(["subject_id", "hadm_id", "icustay_id"])["vasopressor"].agg("max"))
print(outcome.shape)
display(outcome.head())


(37543, 1)


,,,vasopressor
subject_id,hadm_id,icustay_id,
3,145834,211552,1
4,185777,294638,0
6,107064,228232,0
9,150750,220597,1
11,194540,229441,0


In [77]:
vitals_labs_mean = pd.read_hdf(filename, key="vitals_labs_mean")

# Paper name -> df name
# map (mean arterial pressure) -> pulmonary artery pressure mean
# spontaneousrr (spontaneous respiratory rate) -> respiratory rate
# urine (urine output) -> not found! :/
# alt (alanine transaminase) -> alanine aminotransferase
# ast (aspartate aminotransferase) -> asparate aminotransferase
# inr (international normalised ratio) -> prothrombin time inr
mask = vitals_labs_mean.columns.get_level_values("LEVEL2").isin(["diastolic blood pressure", "fraction inspired oxygen", "glascow coma scale total", "heart rate", 
                                                                 "pulmonary artery pressure mean", "systolic blood pressure", "respiratory rate", "oxygen saturation",
                                                                 "temperature", "urine output", "blood urea nitrogen", "magnesium", "platelets", "sodium", "alanine aminotransferase",
                                                                 "hematocrit", "partial pressure of oxygen", "asparate aminotransferase", "potassium", "white blood cell count",
                                                                 "bicarbonate", "creatinine", "lactate", "partial pressure of carbon dioxide", "glucose", "prothrombin time inr",
                                                                 "hemoglobin", "bilirubin"])

# display(vitals_labs_mean.columns[vitals_labs_mean.columns.get_level_values("LEVEL2").str.contains("weight")]) # search column names
vitals_labs_mean = vitals_labs_mean.iloc[:, mask]
vitals_labs_mean = vitals_labs_mean[~vitals_labs_mean.index.get_level_values("subject_id").isin(excluded_subject_ids)]


# use only first 6 hours -> can be replaced by last 6h or random sampling, method not indicated in paper
# TODO random time between LOS_LOWER_THRESH hours_in and 90% quantile of LOS
# CONTROL_ENDTIME_UPPERQUANT = 0.9
# endtimes = np.random.uniform(LOS_LOWER_THRESH, CONTROL_ENDTIME_UPPERQUANT * patients['los_icu'])
vitals_labs_mean = vitals_labs_mean[vitals_labs_mean.index.get_level_values("hours_in").isin(range(6))]
vitals_labs_mean_nan = vitals_labs_mean.copy()


median = vitals_labs_mean.median() # median before or after filling?
vitals_labs_mean = vitals_labs_mean.groupby('subject_id').fillna(method='ffill') # ffill by group over hours_in
vitals_labs_mean = vitals_labs_mean.fillna(median) # fill remaining by median

# normalize
vitals_labs_mean = ( vitals_labs_mean - vitals_labs_mean.mean() ) / vitals_labs_mean.std()

print(vitals_labs_mean.shape)
display(vitals_labs_mean.head(10))


(174822, 27)


LEVEL2                                 alanine aminotransferase  \
Aggregation Function                                       mean   
subject_id hadm_id icustay_id hours_in                            
4          185777  294638     0                           -0.10   
                              1                           -0.10   
                              2                           -0.10   
                              3                           -0.10   
                              4                           -0.10   
                              5                           -0.11   
6          107064  228232     0                           -0.12   
                              1                           -0.12   
                              2                           -0.12   
                              3                           -0.12   

LEVEL2                                 asparate aminotransferase bicarbonate  \
Aggregation Function                                        mean        mean   
subject_id hadm_id icustay_id hours_in                                         
4          185777  294638     0                            -0.09        0.03   
                              1                            -0.09        0.03   
                              2                            -0.09        0.03   
                              3                            -0.09        0.03   
                              4                            -0.09        0.03   
                              5                            -0.05       -0.70   
6          107064  228232     0                            -0.10       -2.17   
                              1                            -0.10       -2.17   
                              2                            -0.10       -2.17   
                              3                            -0.10       -2.17   

LEVEL2                                 bilirubin blood urea nitrogen  \
Aggregation Function                        mean                mean   
subject_id hadm_id icustay_id hours_in                                 
4          185777  294638     0            -0.15               -0.30   
                              1            -0.15               -0.30   
                              2            -0.15               -0.30   
                              3            -0.15               -0.30   
                              4            -0.15               -0.30   
                              5             0.30               -0.72   
6          107064  228232     0            -0.34                2.02   
                              1            -0.34                2.02   
                              2            -0.34                2.02   
                              3            -0.34                2.02   

LEVEL2                                 creatinine diastolic blood pressure  \
Aggregation Function                         mean                     mean   
subject_id hadm_id icustay_id hours_in                                       
4          185777  294638     0             -0.26                -6.54e-02   
                              1             -0.26                 5.77e-03   
                              2             -0.26                 5.77e-03   
                              3             -0.26                -4.21e-01   
                              4             -0.26                -1.36e-01   
                              5             -0.56                -1.36e-01   
6          107064  228232     0              7.73                -4.92e-01   
                              1              7.73                -6.54e-02   
                              2              7.73                -6.54e-02   
                              3              7.73                -3.50e-01   

LEVEL2                                 fraction inspired oxygen  \
Aggregation Function                                       mean   
s

In [78]:
# # Check stuff
# v2 = vitals_labs_mean.reset_index() # before it is truncated
# maxhours = v2.groupby("subject_id")["hours_in"].max()
# display(maxhours.head())
# print(maxhours.shape)

# res = pd.merge(patients, maxhours, on = "subject_id")
# res["los"] = res["los"].astype(int).astype(float) / 3.6e+12 # convert ns to hours
# res["los_icu"] = (res["los_icu"] * 24).astype(int)
# bool = res["hours_in"].equals(res["los_icu"])
# print(f"Hoursin corresponds to los_icu: {bool}")
# display(res[["los", "los_icu", "hours_in"]])
# # => YES

In [79]:
# FORMAT TO TENSOR (N x V x T)

def convert_2d_vitals_labs_to_3d_tensor(data):
    subject_ids = data.index.get_level_values("subject_id").unique()

    N = subject_ids.size
    V = data.shape[1]
    T = 6
    
    # fill 3d array
    tensor = np.zeros((N, V, T))
    for index, row in data.iterrows():
        # display(index)
        # display(row.shape)
        subject_id = index[0]
        hours_in = index[3]
        N_index = subject_ids.get_loc(subject_id)    # map id to index
        
        tensor[N_index, :, hours_in] = row
    return tensor


vitals_labs_mean = convert_2d_vitals_labs_to_3d_tensor(vitals_labs_mean)
vitals_labs_mean_nan = convert_2d_vitals_labs_to_3d_tensor(vitals_labs_mean_nan)
print(vitals_labs_mean.shape)
display(vitals_labs_mean[0,:,0])


(29137, 27, 6)


array([-0.09696495, -0.09444487,  0.0330999 , -0.14676078, -0.2980515 ,
       -0.25999577, -0.06536093, -0.14560194,  0.40155464, -0.17033876,
       -0.05879574, -0.31369481, -0.48568593, -0.04937159, -0.0499835 ,
       -0.86543564, -1.73816138, -0.06106983, -0.16638308, -0.06468431,
       -0.14332714, -0.01382433, -0.04658971,  0.10619543, -0.08300108,
       -0.01689232, -0.12864698])

In [80]:
indicators = ~np.isnan(vitals_labs_mean_nan)
print(indicators.shape)
display(indicators[0,:,0])


(29137, 27, 6)


array([False, False, False, False, False, False, False, False, False,
       False, False,  True,  True,  True, False,  True,  True, False,
       False, False, False, False, False, False, False, False, False])

In [81]:
print(f"Exclude {excluded_subject_ids.size} patients...")

patients = patients[~patients.index.get_level_values("subject_id").isin(excluded_subject_ids)]
print(patients.shape)

outcome = outcome[~outcome.index.get_level_values("subject_id").isin(excluded_subject_ids)]
print(outcome.shape)

# vitals_labs_mean = vitals_labs_mean[~vitals_labs_mean.index.get_level_values("subject_id").isin(excluded_subject_ids)]
print(vitals_labs_mean.shape)

# indicators = indicators[~indicators.index.get_level_values("subject_id").isin(excluded_subject_ids)]
print(indicators.shape)

Exclude 8406 patients...
(29137, 8)
(29137, 1)
(29137, 27, 6)
(29137, 27, 6)


In [82]:
current_time = datetime.datetime.now().strftime("%Y_%m_%d__%H_%M")

OUTPUT_PATH = f"./{current_time}-vasopressor-"

print(OUTPUT_PATH)

patients.to_pickle(OUTPUT_PATH + 'patients.p')
outcome.to_pickle(OUTPUT_PATH + 'outcome.p')

np.save(OUTPUT_PATH + 'vitals_labs_mean.npy', vitals_labs_mean)
np.save(OUTPUT_PATH + 'indicators.npy', indicators)

# LOAD
# patients = pd.read_pickle("./2023_10_19__12_49-vasopressor-patients.npy")
# outcome = pd.read_pickle("./2023_10_19__12_49-vasopressor-outcome.npy")
# vitals_labs_mean = np.load("./2023_10_19__12_49-vasopressor-vitals_labs_mean.npy")
# indicators = np.load("./2023_10_19__12_49-vasopressor-indicators.npy")

print('Dumped')

./2023_10_21__13_25-vasopressor-
Dumped
